In [ ]:
from langchain_community.utilities import WikipediaAPIWrapper, DuckDuckGoSearchAPIWrapper
from langchain_core.tools import tool

wikipedia_tool = WikipediaAPIWrapper()
web_search_tool = DuckDuckGoSearchAPIWrapper()

@tool
def buscar_informacion_persona(nombre: str) -> str:
    """Get information of a specific 'name' given in the web."""
    try:
        info_wikipedia = wikipedia_tool.run(nombre)
        if info_wikipedia:
            print("Información obtenida de Wikipedia:")
            print(info_wikipedia)
            return info_wikipedia
        else:
            print("No se encontró información en Wikipedia.")
    except Exception as e:
        print(f"Error al buscar en Wikipedia: {e}")

    print("Intentando buscar en la web...")





In [ ]:
from pydantic import BaseModel
from langchain_core.tools import tool

class ExtractPersonalDataArgs(BaseModel):
    textos: list
    nombre_persona: str

@tool
def extraer_datos_personales_openai(data):
    """ Based on a json with keys, textos of type List and nombre_persona of type string, extract the news and nombre_persona and get the personal data of a specific person based on the news List  """
    data = json.loads(data)
    textos = data.get('textos')
    nombre_persona = data.get('nombre_persona')
    
    texto_concatenado = " ".join(textos)
    prompt_template = """
        Extrae información personal sobre {nombre_persona} de este texto y escribe un resumen en forma de párrafo.
        Incluye, si están presentes, el nombre completo, fecha de nacimiento, lugar de residencia, datos de contacto, ocupación, logros, relaciones relevantes y cualquier cita textual significativa.
        Si no hay información relevante sobre {nombre_persona}, responde 'No se encontró información personal relevante'.
        Texto:
        {texto_concatenado}"
    """
    prompt = ChatPromptTemplate.from_template(prompt_template)
    formatted_prompt = prompt.format_messages(texto_concatenado=texto_concatenado, nombre_persona=nombre_persona)
    response = chat(formatted_prompt).content

  
    return response


In [ ]:
import spacy
from difflib import SequenceMatcher
from langchain_core.tools import tool

def es_similar(nombre1, nombre2, umbral=0.8):
    return SequenceMatcher(None, nombre1, nombre2).ratio() >= umbral

@tool
def filtrar_noticias_por_persona(data):
    """ Get of input a dictionary with keys 'noticias' and 'nombre_persona' and based on that get the news what are related with a specific user. """
    data = json.loads(data)
    noticias = data.get('noticias')
    nombre_persona = data.get('nombre_persona')


    nlp = spacy.load("es_core_news_md")
    nombre_persona = nombre_persona.lower()
    noticias_filtradas = []

    for noticia in noticias:
        doc = nlp(noticia)
        for ent in doc.ents:
            if ent.label_ == "PER" and es_similar(nombre_persona, ent.text.lower()):
                noticias_filtradas.append(noticia)
                break

    return noticias_filtradas



In [ ]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.llms import OpenAI
from langchain_core.tools import tool

@tool
def contar_personas_mencionadas(data):
    """Get a dict with key 'noticias', and return the count of persons in the given list of 'noticias'. """
    data = json.loads(data)
    noticias = data.get('noticias')
    personas = []
    for noticia in noticias:
        prompt = PromptTemplate(
            input_variables=["texto"],
            template="Extrae todos los nombres de personas mencionadas en el siguiente texto: {texto}"
        )
        llm_chain = LLMChain(llm=OpenAI(), prompt=prompt)
        personas_encontradas = llm_chain.run(noticia)  
        personas.extend(personas_encontradas.split(", "))
    return len(set(personas))



In [ ]:
import json
from datetime import datetime
from langchain.prompts import PromptTemplate
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate
from langchain_core.tools import tool

@tool
def obtener_datos_consulta(texto):
    """Given a text return a dictionary with the topic of the news, and the start_date and the end_date. """
    template = """
    Extrae la temática, la fecha de inicio y la fecha de fin del siguiente texto. 
            El formato de la fecha, tanto inicio como fin debe ser el siguiente: Y-m-d. 
                Devuelve los resultados en formato JSON con las claves: tematica, fecha_inicio, fecha_fin
                en caso de no existir alguno de los valores, deja los campos como vacios.
                Texto: {texto}
    """
    prompt = ChatPromptTemplate.from_template(template)
    formatted_prompt = prompt.format_messages(texto=texto, pregunta="¿Qué es el aprendizaje profundo?")
    response = chat(formatted_prompt).content
    response_dict = json.loads(response)
    return response_dict

# datos = obtener_datos_consulta("Cuales son las personas que más se mencionan entre dos fechas sobre la temática del hidrogeno verde? ")
# chroma_db_data = buscar_en_chromadb(datos)
# print(chroma_db_data)
